## Imports

In [1]:
import pandas as pd
import numpy as np
import datetime
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble  import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import mean_squared_error
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from math import sqrt
from sklearn.metrics import accuracy_score
# from dotenv import load_dotenv, find_dotenv
# load_dotenv(find_dotenv())

## Leitura e ajuste dos Datasets

In [2]:
# Definindo o caminho dos dados
data_company_path = "dados/Company.csv"
data_tweet_path = "dados/Tweet.csv"
data_company_tweet_path = "dados/Company_Tweet.csv"

In [3]:
# Ler e colocar em um dataframe
data_company = pd.read_csv(data_company_path)
data_tweet = pd.read_csv(data_tweet_path)
data_company_tweet = pd.read_csv(data_company_tweet_path)

In [4]:
print(data_company_tweet.head())
print(data_company_tweet.shape)

             tweet_id ticker_symbol
0  550803612197457920          AAPL
1  550803610825928706          AAPL
2  550803225113157632          AAPL
3  550802957370159104          AAPL
4  550802855129382912          AAPL
(4336445, 2)


In [5]:
print(data_company)
print(data_company.shape)

  ticker_symbol company_name
0          AAPL        apple
1          GOOG   Google Inc
2         GOOGL   Google Inc
3          AMZN   Amazon.com
4          TSLA    Tesla Inc
5          MSFT    Microsoft
(6, 2)


In [6]:
print(data_tweet.head())
print(data_tweet.shape)

             tweet_id           writer   post_date  \
0  550441509175443456  VisualStockRSRC  1420070457   
1  550441672312512512      KeralaGuy77  1420070496   
2  550441732014223360      DozenStocks  1420070510   
3  550442977802207232     ShowDreamCar  1420070807   
4  550443807834402816     i_Know_First  1420071005   

                                                body  comment_num  \
0  lx21 made $10,008  on $AAPL -Check it out! htt...            0   
1  Insanity of today weirdo massive selling. $aap...            0   
2  S&P100 #Stocks Performance $HD $LOW $SBUX $TGT...            0   
3  $GM $TSLA: Volkswagen Pushes 2014 Record Recal...            0   
4  Swing Trading: Up To 8.91% Return In 14 Days h...            0   

   retweet_num  like_num  
0            0         1  
1            0         0  
2            0         0  
3            0         1  
4            0         1  
(3717964, 7)


In [7]:
dados_merge1 = pd.merge(data_company_tweet,data_company, on='ticker_symbol')
dados_merge1 = dados_merge1.drop(columns=['ticker_symbol'])
dados_merge1.head()

,tweet_id,company_name
0,550803612197457920,apple
1,550803610825928706,apple
2,550803225113157632,apple
3,550802957370159104,apple
4,550802855129382912,apple


In [8]:
dados_tweet_company = pd.merge(dados_merge1,data_tweet,on='tweet_id')
dados_tweet_company = dados_tweet_company.drop(columns=['tweet_id'])
dados_tweet_company.head()

,company_name,writer,post_date,body,comment_num,retweet_num,like_num
0,apple,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1
1,Amazon.com,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1
2,apple,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1
3,Google Inc,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1
4,Amazon.com,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1


In [9]:
#dados_tweet_company = dados_tweet_company.iloc[:500000]
#dados_tweet_company.head()

In [10]:
dados_tweet_company.shape

(4336445, 7)

## Ajustar Tempo (Data do Tweet)

In [11]:
dados_tweet_company.post_date[0]

1420156789

In [12]:
ts_epoch = dados_tweet_company.post_date[0]
ts = datetime.datetime.fromtimestamp(ts_epoch).strftime('%Y-%m-%d %H:%M:%S')
ts

'2015-01-01 20:59:49'

In [13]:
def ajustar_tempo(row):
    ts_epoch = row['post_date']
    ts = datetime.datetime.fromtimestamp(ts_epoch).strftime('%Y-%m-%d %H:%M:%S')
    return ts  
new_date = dados_tweet_company.apply(ajustar_tempo,axis=1)
dados_tweet_company['Date'] = list(new_date)
dados_tweet_company.head()

,company_name,writer,post_date,body,comment_num,retweet_num,like_num,Date
0,apple,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1,2015-01-01 20:59:49
1,Amazon.com,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1,2015-01-01 20:59:49
2,apple,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48
3,Google Inc,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48
4,Amazon.com,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48


In [14]:
dados_tweet_company.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4336445 entries, 0 to 4336444
Data columns (total 8 columns):
company_name    object
writer          object
post_date       int64
body            object
comment_num     int64
retweet_num     int64
like_num        int64
Date            object
dtypes: int64(4), object(4)
memory usage: 297.8+ MB


## Removendo Caracteres 

In [15]:
sentenca = dados_tweet_company.body[0]
print(sentenca)

sentenca_2 = dados_tweet_company.body[100]
print(sentenca_2)

#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN $T $IGN $SPY $FB $UGAZ #sentiquant 20150101 19:00:08:298
Our Penny Stock Picks Gained Over 968% In the Past 5 Weeks! Get our next pick early: http://tinyurl.com/p46mnk2 $TRX $AAPL $GOOG


In [16]:
def replace_all(texto):
    palavras_especiais = [word for word in texto.split(' ') if word.startswith(('#','$','@'))]
    #print(palavras_especiais)
    sentenca = ' '.join(x for x in texto.split(' ') if x not in palavras_especiais)
    return sentenca

new_strings = dados_tweet_company.body.apply(replace_all)
dados_tweet_company['Processed_Tweet'] = list(new_strings)
dados_tweet_company.head()

,company_name,writer,post_date,body,comment_num,retweet_num,like_num,Date,Processed_Tweet
0,apple,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1,2015-01-01 20:59:49,20150101 19:00:08:298
1,Amazon.com,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1,2015-01-01 20:59:49,20150101 19:00:08:298
2,apple,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48,20150101 19:00:04:526
3,Google Inc,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48,20150101 19:00:04:526
4,Amazon.com,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48,20150101 19:00:04:526


In [17]:
sentenca = dados_tweet_company.Processed_Tweet[0]
print(sentenca)

sentenca_2 = dados_tweet_company.Processed_Tweet[100]
print(sentenca_2)


20150101 19:00:08:298
Our Penny Stock Picks Gained Over 968% In the Past 5 Weeks! Get our next pick early: http://tinyurl.com/p46mnk2


## Filtrando Dados

In [18]:
#dados_tweet_company['Date'] = pd.to_datetime(dados_tweet_company['Date'])
#dados_tweet_company = dados_tweet_company[dados_tweet_company['Date'].dt.year == 2019]
#dados_tweet_company.head()

In [19]:
#dados_tweet_company = dados_tweet_company[dados_tweet_company.company_name == 'Microsoft']
#print(dados_tweet_company.shape)
#dados_tweet_company.head()

## Criando Labels para os Tweets 
* Positivo = 1 or pos
* Neutro = 0 or neu
* Negativo = -1 or neg

In [20]:
#'https://towardsdatascience.com/sentimental-analysis-using-vader-a3415fef7664'

In [21]:
sid = SentimentIntensityAnalyzer()
sid.polarity_scores(dados_tweet_company.body[1541910])

{'neg': 0.0, 'neu': 0.959, 'pos': 0.041, 'compound': 0.2023}

In [22]:
dados_tweet_company.shape

(4336445, 9)

In [23]:
dados_tweet_company['vader_scores']  = None
dados_tweet_company['vader_scores'] = dados_tweet_company.Processed_Tweet.apply(lambda tweet : sid.polarity_scores(tweet))
dados_tweet_company.head()

,company_name,writer,post_date,body,comment_num,retweet_num,like_num,Date,Processed_Tweet,vader_scores
0,apple,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1,2015-01-01 20:59:49,20150101 19:00:08:298,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
1,Amazon.com,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1,2015-01-01 20:59:49,20150101 19:00:08:298,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
2,apple,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48,20150101 19:00:04:526,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
3,Google Inc,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48,20150101 19:00:04:526,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
4,Amazon.com,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48,20150101 19:00:04:526,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."


In [24]:
# separar os valores do compound - vamos uas-lo para determinar os lbels dos tweets pos/beg
dados_tweet_company['compound']  = dados_tweet_company['vader_scores'].apply(lambda score_dict: score_dict['compound'])
dados_tweet_company.head()

,company_name,writer,post_date,body,comment_num,retweet_num,like_num,Date,Processed_Tweet,vader_scores,compound
0,apple,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1,2015-01-01 20:59:49,20150101 19:00:08:298,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0
1,Amazon.com,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1,2015-01-01 20:59:49,20150101 19:00:08:298,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0
2,apple,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48,20150101 19:00:04:526,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0
3,Google Inc,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48,20150101 19:00:04:526,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0
4,Amazon.com,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48,20150101 19:00:04:526,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0


In [25]:
# definindo os labels usando o compund value como referenecia
def labels(value):
    if value == 0:
        return 'neu'
    elif value > 0:
        return 'pos'
    elif value < 0:
        return 'neg'
    
dados_tweet_company['label']  = dados_tweet_company['compound'].apply(lambda value: labels(value))
dados_tweet_company.head()

,company_name,writer,post_date,body,comment_num,retweet_num,like_num,Date,Processed_Tweet,vader_scores,compound,label
0,apple,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1,2015-01-01 20:59:49,20150101 19:00:08:298,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,neu
1,Amazon.com,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1,2015-01-01 20:59:49,20150101 19:00:08:298,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,neu
2,apple,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48,20150101 19:00:04:526,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,neu
3,Google Inc,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48,20150101 19:00:04:526,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,neu
4,Amazon.com,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48,20150101 19:00:04:526,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,neu


In [26]:
dados_tweet_company.label.value_counts()

neu    1883920
pos    1703413
neg     749112
Name: label, dtype: int64

## Dados para ML
* Processed_Tweet - feature
* Label - target

In [27]:
# separando os valores para a machine learning
dados = dados_tweet_company[['Processed_Tweet','label']]
dados.head()

,Processed_Tweet,label
0,20150101 19:00:08:298,neu
1,20150101 19:00:08:298,neu
2,20150101 19:00:04:526,neu
3,20150101 19:00:04:526,neu
4,20150101 19:00:04:526,neu


## Balanceando Dados

In [28]:
dados.label.value_counts()

neu    1883920
pos    1703413
neg     749112
Name: label, dtype: int64

In [29]:
# filtrar dados por fase
dataset_pos = dados[dados['label']=='pos']
dataset_neu = dados[dados['label']=='neu']
dataset_neg = dados[dados['label']=='neg']

print(f'Tamanho POS: {len(dataset_pos)}')
print(f'Tamanho NEU: {len(dataset_neu)}')
print(f'Tamanho NEG: {len(dataset_neg)}')

print('#'*100)
dataset_tamanhos = [len(dataset_pos),len(dataset_neu),len(dataset_neg)]
print('Tamanhos: ',dataset_tamanhos)
print('#'*100)

tamanho_min = min(dataset_tamanhos)
print(tamanho_min)

Tamanho POS: 1703413
Tamanho NEU: 1883920
Tamanho NEG: 749112
####################################################################################################
Tamanhos:  [1703413, 1883920, 749112]
####################################################################################################
749112


In [30]:
data_a = dataset_pos.iloc[:tamanho_min-1].copy()
data_b = dataset_neu.iloc[:tamanho_min-1].copy()
data_c = dataset_neg.iloc[:tamanho_min-1].copy()

frames = [data_a, data_b,data_c]

df_data = pd.concat(frames)

In [31]:
print(df_data.shape)
df_data.head()

(2247333, 2)


,Processed_Tweet,label
6,"An store line in Sapporo Japan for the ""Lucky ...",pos
7,- Will Give Second entry opportunity? - https:...,pos
8,Will Give Second entry opportunity? http://tra...,pos
9,We will book gains in half dozen picks tomorro...,pos
16,"""@CNBC: This is Wall Street's top pick in 2015...",pos


## Criando conjuntos de Treino e Teste

In [32]:
datasetVariables_train, datasetVariables_test, datasetTarget_train, datasetTarget_test = train_test_split(df_data['Processed_Tweet'], df_data['label'], test_size=0.33, random_state=42)

## Vetorizando Tweets
* Tokenizer - Tensorflow Keras

In [33]:
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(datasetVariables_train)

word_index = tokenizer.word_index
print(len(word_index))

646280


In [35]:
#print(word_index)

## Padding Vetores 
* Tamanho max dos vetores = 300

In [36]:
# Transformar as sentencas dos dados de treinamento e teste para sequencias numericas. ex [[1,2,6],[65,952,15,65],[1,2]]
train_data_sentences = tokenizer.texts_to_sequences(datasetVariables_train)
test_data_sentences = tokenizer.texts_to_sequences(datasetVariables_test)

# normalizar o tamnho das sequencias usando padding. ex [[0,1,2,6],[65,952,15,65],[0,0,1,2]]
train_padded = pad_sequences(train_data_sentences, maxlen=300)
test_padded = pad_sequences(test_data_sentences, maxlen=300)

print("Conjunto de treinamento: ","\n",train_padded[0:19])
print("Conjunto de teste: ","\n",test_padded[0:19])

Conjunto de treinamento:  
 [[     0      0      0 ...     19     15 112004]
 [     0      0      0 ...     19     15 195539]
 [     0      0      0 ...    137   6495    450]
 ...
 [     0      0      0 ...    126    278     61]
 [     0      0      0 ...     83     41     91]
 [     0      0      0 ... 195545     61    116]]
Conjunto de teste:  
 [[     0      0      0 ...     99   1540    127]
 [     0      0      0 ...     55     16      1]
 [     0      0      0 ...     27      6   3746]
 ...
 [     0      0      0 ...      1  22855      7]
 [     0      0      0 ...      5 497845    116]
 [     0      0      0 ...      5   2713   4983]]


In [37]:
datasetTarget_test[:15]

298216     neu
452630     neu
149558     neg
915791     pos
2828417    neg
1089875    neu
584643     pos
394869     neu
1748568    pos
1943215    neg
1786346    pos
976544     neu
2654352    neg
915599     pos
1329220    pos
Name: label, dtype: object

In [ ]:
#datasetTarget_train = to_categorical(np.array(datasetTarget_train))
#datasetTarget_test = to_categorical(np.array(datasetTarget_test))

## Machine Learning 
* Random forest

In [38]:
# first ML model
model_randomForest = RandomForestClassifier(random_state=50,criterion='gini')
model_randomForest.fit(train_padded,datasetTarget_train)

RandomForestClassifier(random_state=50)

In [39]:
predicaoTrain = model_randomForest.predict(train_padded)
predicaoTest = model_randomForest.predict(test_padded)

print("ACC TRAINING:" + str(accuracy_score(datasetTarget_train, predicaoTrain)))
print("ACC TEST:" + str(accuracy_score(datasetTarget_test, predicaoTest)))

ACC TRAINING:0.999853889818312
ACC TEST:0.7652261265877404


In [40]:
from sklearn.metrics import classification_report
print(classification_report(datasetTarget_test, predicaoTest))

              precision    recall  f1-score   support

         neg       0.67      0.85      0.75    247017
         neu       0.82      0.77      0.80    247437
         pos       0.86      0.67      0.76    247166

    accuracy                           0.77    741620
   macro avg       0.78      0.77      0.77    741620
weighted avg       0.78      0.77      0.77    741620



In [41]:
datasetTarget_test.value_counts()

neu    247437
pos    247166
neg    247017
Name: label, dtype: int64

In [42]:
from sklearn.metrics import confusion_matrix
confusion_matrix(datasetTarget_test, predicaoTest, labels=["neu", "pos", "neg"])

array([[191598,  12978,  42861],
       [ 18575, 166699,  61892],
       [ 23094,  14713, 209210]], dtype=int64)

In [43]:
from sklearn.naive_bayes import MultinomialNB
model_naiveBayes = MultinomialNB()
model_naiveBayes.fit(train_padded,datasetTarget_train)

C:\Users\Usuário\AppData\Roaming\Python\Python37\site-packages\sklearn\naive_bayes.py:780: RuntimeWarning: invalid value encountered in log
  self.feature_log_prob_ = (np.log(smoothed_fc) -


MultinomialNB()

In [44]:
predicaoTrain = model_naiveBayes.predict(train_padded)
predicaoTest = model_naiveBayes.predict(test_padded)

print("ACC TRAINING:" + str(accuracy_score(datasetTarget_train, predicaoTrain)))
print("ACC TEST:" + str(accuracy_score(datasetTarget_test, predicaoTest)))

from sklearn.metrics import classification_report
print(classification_report(datasetTarget_test, predicaoTest))

ACC TRAINING:0.33345929802027346
ACC TEST:0.3330775869043445


C:\Users\Usuário\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Usuário\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         neg       0.33      1.00      0.50    247017
         neu       0.00      0.00      0.00    247437
         pos       0.00      0.00      0.00    247166

    accuracy                           0.33    741620
   macro avg       0.11      0.33      0.17    741620
weighted avg       0.11      0.33      0.17    741620



C:\Users\Usuário\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [45]:
from sklearn.linear_model import SGDClassifier
model_linear = SGDClassifier()
model_linear.fit(train_padded,datasetTarget_train)

predicaoTrain = model_linear.predict(train_padded)
predicaoTest = model_linear.predict(test_padded)

print("ACC TRAINING:" + str(accuracy_score(datasetTarget_train, predicaoTrain)))
print("ACC TEST:" + str(accuracy_score(datasetTarget_test, predicaoTest)))

from sklearn.metrics import classification_report
print(classification_report(datasetTarget_test, predicaoTest))

C:\Users\Usuário\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\_stochastic_gradient.py:577: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


ACC TRAINING:0.35654869155011615
ACC TEST:0.3597260052317899
              precision    recall  f1-score   support

         neg       0.41      0.29      0.34    247017
         neu       0.33      0.19      0.24    247437
         pos       0.35      0.60      0.44    247166

    accuracy                           0.36    741620
   macro avg       0.36      0.36      0.34    741620
weighted avg       0.36      0.36      0.34    741620



In [46]:
from sklearn.linear_model import LogisticRegression
model_logisticRegression = LogisticRegression()
model_logisticRegression.fit(train_padded,datasetTarget_train)

predicaoTrain = model_logisticRegression.predict(train_padded)
predicaoTest = model_logisticRegression.predict(test_padded)

print("ACC TRAINING:" + str(accuracy_score(datasetTarget_train, predicaoTrain)))
print("ACC TEST:" + str(accuracy_score(datasetTarget_test, predicaoTest)))

from sklearn.metrics import classification_report
print(classification_report(datasetTarget_test, predicaoTest))

C:\Users\Usuário\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


MemoryError: Unable to allocate 3.37 GiB for an array with shape (1505713, 300) and data type float64

In [ ]:
# save model
#import pickle
#filename = 'model2_randomForest_80.sav'
#pickle.dump(model_v1,open(filename,'wb'))

In [ ]:
from sklearn import svm
from sklearn.svm import LinearSVC
model_SVM = svm.SVC(gamma='auto')
model_SVM.fit(train_padded,datasetTarget_train)

In [ ]:
predicaoTrain = model_SVM.predict(train_padded)
predicaoTest = model_SVM.predict(test_padded)

print("ACC TRAINING:" + str(accuracy_score(datasetTarget_train, predicaoTrain)))
print("ACC TEST:" + str(accuracy_score(datasetTarget_test, predicaoTest)))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(datasetTarget_test, predicaoTest))

In [ ]:
mat = confusion_matrix(datasetTarget_test, predicaoTest)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False)
plt.xlabel('true label')
plt.ylabel('predicted label');

In [ ]:
estimator = []
estimator.append(('SVC', SVC(probability = True)))
estimator.append(('RND', RandomForestClassifier()))
  
# Voting Classifier with hard voting
vot_hard = VotingClassifier(estimators = estimator, voting ='hard')
vot_hard.fit(X_train, y_train)
y_pred = vot_hard.predict(X_test)
  
# using accuracy_score metric to predict accuracy
score = accuracy_score(y_test, y_pred)
print("Hard Voting Score % d" % score)
  
# Voting Classifier with soft voting
vot_soft = VotingClassifier(estimators = estimator, voting ='soft')
vot_soft.fit(X_train, y_train)
y_pred = vot_soft.predict(X_test)
  
# using accuracy_score
score = accuracy_score(y_test, y_pred)
print("Soft Voting Score % d" % score)